# "My Jupyter Notebook on IBM Data Science Experience"

**William Morgan** <br>
Any software engineering/data science position


*I am interested in data science, because i love compute power and numbers. I love answering questions hidden in data and uncovering other insights that were not even considered in the first place. I am a very curious person and love using data to answer questions and prove my case.*

### In the next cell, you will see code that scrapes the first page of results from a website I have recently been working on. It puts the data into a pandas DataFrame and displays the first 15 columns of the first 5 results.

In [4]:
import numpy as np
import pandas as pd

import re
import time

from bs4 import BeautifulSoup
import requests

start_time = time.time()

page_n = 1 #keep track of which page the scraper is on
games = []# list to collect data
base_url = 'https://boardgamegeek.com/' #the base url
start_url = 'https://boardgamegeek.com/browse/boardgame/page/' #start page

# a list of attributes to be collected from 'all_d'
collect_list = ["objectid","name","yearpublished","sortindex","minplayers",
                "maxplayers","minplaytime","maxplaytime","minage","best",
                "max","totalvotes","playerage","languagedependence", 
                "usersrated","average","baverage","stddev","avgweight", 
                "numweights","numgeeklists","numtrading","numwanting",
                "numcomments","views","numplays","numplays_month","news",
                "blogs","weblink","podcast","label","boardgamedesigner",
                "boardgameartist","boardgamepublisher","boardgamehonor",
                "boardgamecategory","boardgamemechanic","boardgameexpansion",
                "boardgameversion","boardgamefamily"]
#a list of attributes to be collected from "credit_d"
credit_collect_list = ["boardgamedesigner","boardgameartist","boardgamepublisher",
                       "boardgamehonor","boardgamecategory","boardgameversion","boardgamemechanic"]
# while loop to continue until page_n reached the decided limit
while page_n <= 1 :
    #request the page
    game_names = BeautifulSoup(requests.get(start_url+str(page_n)).text,'lxml')
    #get list of games from page
    game_list = game_names.find_all('td',{'class':'collection_objectname'}) 
    
    #below is logic to stop the loop if no additional links are available
    if not game_list:
        print('No more results: Exiting...') 
        break
    else:
        print('Getting page:{}'.format(page_n))
        
        #iterate through game list for page_n
        for game in game_list:
            game_data = [] #collect data
            #request game page
            game_page = BeautifulSoup(requests.get(base_url+\
                        game.select('a')[0]['href']+'/stats').text,'html.parser')
            #capture whole description of game
            desc_tag = game_page.find_all('meta',{'property':'og:description'})
            desc_str = str(desc_tag).split('>',1)[0][16:-27].replace('&amp;ldquo;','"')\
            .replace('&amp;rdquo;','"').replace('\n',' ')
            
            #manipulate js script data and create dictionaries for attribute look up
            script =game_page.find("script", text=re.compile("GEEK.geekitemPreload\s+="))
            data_list = str(script).replace('=4&mt=8&at=','').replace(':{"link"','')\
            .split("=",9)[9].strip().split(':{')
            
            comm_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[3][:-1].split(',') if len(i.split(':'))> 1 }
            stats_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                       for i in data_list[5][:-1].split(',') if len(i.split(':'))> 1 }
            counts_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                        for i in data_list[6][:-1].split(',') if len(i.split(':'))> 1 }
            info_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[7][:-1].split(',') if len(i.split(':'))> 1 }
            cr_cnt_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                        for i in data_list[9][:-1].split(',') if len(i.split(':'))> 1 }
            rank_d = {i.split(':')[0]:' '.join(i.split(':')[1:])\
                      for i in data_list[10][:-1].split(',') if len(i.split(':'))> 1 }
            
            all_d = {**comm_d, **stats_d,**counts_d,**info_d,**cr_cnt_d,**rank_d}
            
            #extract data from messy string
            credit_d = {}
            for i in data_list[8].split('],'):
                item = i.split(":",1)
                try:
                    values = [h.split(',')[0] for h in\
                              [j.split(':')[1] for j in item[1].split('},{')]]
                except:
                    pass
                
                credit_d[item[0]]=values
                
            #iterate through list of desired attributes to capture values
            for i in collect_list:
                if '"'+i+'"' in all_d.keys():
                    game_data.append(all_d['"'+i+'"'].replace('"',''))
                else:
                    game_data.append('NaN')
            for i in credit_collect_list:
                if '"'+i+'"' in credit_d.keys():
                    game_data.append(credit_d['"'+i+'"'])
                else:
                    game_data.append('NaN')
            
            game_data.append(desc_str) #append game description
            game_data.append(game.select('a')[0]['href'])
            games.append(game_data)
        page_n += 1
        
end_time = time.time()

print('total_time: {} minutes'.format((end_time-start_time)/60))

df = pd.DataFrame.from_records(games)
df.iloc[:,:15].head()

Getting page:1
total_time: 0.10658369461695354 minutes


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,174430,Gloomhaven,2017,1,1,4,60,120,12,[{min 3,4}],825},14+,Extensive use of text - massive conversion nee...,31176
1,161936,Pandemic Legacy Season 1,2015,1,2,4,60,60,13,[{min 4,4}],548},12+,Extensive use of text - massive conversion nee...,34681
2,167791,Terraforming Mars,2016,1,1,5,120,120,12,[{min 3,4}],1250},12+,Moderate in-game text - needs crib sheet or pa...,48216
3,182028,Through the Ages A New Story of Civilization,2015,1,2,4,120,120,14,[{min 3,4}],436},14+,Extensive use of text - massive conversion nee...,18238
4,224517,Brass Birmingham,2018,1,2,4,60,120,14,[{min 3,4}],240},14+,No necessary in-game text,10021


 1. an 
 2. Ordered
 3. List
 
  - An
  - Unordered
  - subList <br>
```python
x = 'this is inline python code'
```